#### An experimentation with multiple choice questions created using [stanza's](https://stanfordnlp.github.io/stanza/) Ancient [Greek](https://stanfordnlp.github.io/stanza/available_models.html) parser and [jtauber's](https://github.com/jtauber) accentuation rules

---
## Resources
greek-accentuation [documentation](https://github.com/jtauber/greek-accentuation/blob/master/docs.rst)

greek-normalization [documentation](https://github.com/jtauber/greek-normalisation/blob/master/tests.rst)

---
---

## Installations

In [11]:
!pip install stanza
!pip install gradio
!pip install greek-accentuation==1.2.0
!pip install greek-normalisation
from greek_accentuation.syllabify import *
from greek_accentuation.characters import *
# from greek_accentuation.accentuation import * 
from lib.greek_accentuation.greek_accentuation.accentuation import *
from greek_normalisation.utils import *
import gradio as gr
import random

---
---
## Generate Exercises

---
### Exercise 1: Find the main verb of the sentence

    Step 0: Get quiz questions

In [169]:
# define the file name here:
quiz = 'lib/quiz_questions.txt'

# list to hold each line of the file
lines = []
# list of dictionaries for holding the English answer/ Greek answer
exercises = []

# Read in the lines from the file
with open(quiz) as f:
    # create list for holding the exercises
    lines = f.readlines()

# For each line, use regex to grab the answer and full sentence
for sent in lines:
    
    # Get the greek answer
    eng_ans_end = sent.find(':')
    english_answer = sent[0:eng_ans_end]

    greek_answer = sent[eng_ans_end+1:]
    
    # Add everything to our list of dictionaries
    exercises.append({"english answer":english_answer, "greek answer":greek_answer})
    

    Step 1: Parse the sentence

In [170]:
# define the sentence
sentence = "τῷ στρατηγῷ πέμπει τοὺς ἀδελφούς"

In [171]:
import stanza

# stanza.download('grc') 
nlp = stanza.Pipeline('grc') 
doc = nlp(sentence) 

print(doc)
print(doc.entities)

2022-06-29 12:36:42 INFO: Loading these models for language: grc (Ancient_Greek):
| Processor | Package |
-----------------------
| tokenize  | proiel  |
| pos       | proiel  |
| lemma     | proiel  |
| depparse  | proiel  |

2022-06-29 12:36:42 INFO: Use device: cpu
2022-06-29 12:36:42 INFO: Loading: tokenize
2022-06-29 12:36:42 INFO: Loading: pos
2022-06-29 12:36:42 INFO: Loading: lemma
2022-06-29 12:36:42 INFO: Loading: depparse
2022-06-29 12:36:43 INFO: Done loading processors!


[
  [
    {
      "id": 1,
      "text": "τῷ",
      "lemma": "ὁ",
      "upos": "DET",
      "xpos": "S-",
      "feats": "Case=Dat|Definite=Def|Gender=Masc|Number=Sing|PronType=Dem",
      "head": 2,
      "deprel": "det",
      "start_char": 0,
      "end_char": 2
    },
    {
      "id": 2,
      "text": "στρατηγῷ",
      "lemma": "στρατηγός",
      "upos": "NOUN",
      "xpos": "Nb",
      "feats": "Case=Dat|Gender=Masc|Number=Sing",
      "head": 3,
      "deprel": "iobj",
      "start_char": 3,
      "end_char": 11
    },
    {
      "id": 3,
      "text": "πέμπει",
      "lemma": "πέμπω",
      "upos": "VERB",
      "xpos": "V-",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act",
      "head": 0,
      "deprel": "root",
      "start_char": 12,
      "end_char": 18
    },
    {
      "id": 4,
      "text": "τοὺς",
      "lemma": "ὁ",
      "upos": "DET",
      "xpos": "S-",
      "feats": "Case=Acc|Definite=Def|Gender=Masc|Number=Plur|PronType=Dem"

    Step 2: Find the root of the sentence

In [172]:
root = [word for sent in doc.sentences for word in sent.words if word.deprel=="root"]
print(root)
not_roots = [word for sent in doc.sentences for word in sent.words if word.deprel!="root"]
print(not_roots)

[{
  "id": 3,
  "text": "πέμπει",
  "lemma": "πέμπω",
  "upos": "VERB",
  "xpos": "V-",
  "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act",
  "head": 0,
  "deprel": "root",
  "start_char": 12,
  "end_char": 18
}]
[{
  "id": 1,
  "text": "τῷ",
  "lemma": "ὁ",
  "upos": "DET",
  "xpos": "S-",
  "feats": "Case=Dat|Definite=Def|Gender=Masc|Number=Sing|PronType=Dem",
  "head": 2,
  "deprel": "det",
  "start_char": 0,
  "end_char": 2
}, {
  "id": 2,
  "text": "στρατηγῷ",
  "lemma": "στρατηγός",
  "upos": "NOUN",
  "xpos": "Nb",
  "feats": "Case=Dat|Gender=Masc|Number=Sing",
  "head": 3,
  "deprel": "iobj",
  "start_char": 3,
  "end_char": 11
}, {
  "id": 4,
  "text": "τοὺς",
  "lemma": "ὁ",
  "upos": "DET",
  "xpos": "S-",
  "feats": "Case=Acc|Definite=Def|Gender=Masc|Number=Plur|PronType=Dem",
  "head": 5,
  "deprel": "det",
  "start_char": 19,
  "end_char": 23
}, {
  "id": 5,
  "text": "ἀδελφούς",
  "lemma": "ἀδελφός",
  "upos": "NOUN",
  "xpos": "Nb",
  "feats": 

In [173]:
# get the lemmas for each word
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

word: τῷ 	lemma: ὁ
word: στρατηγῷ 	lemma: στρατηγός
word: πέμπει 	lemma: πέμπω
word: τοὺς 	lemma: ὁ
word: ἀδελφούς 	lemma: ἀδελφός


    Step 3: Generate other, incorrect answers

In [174]:
wrong_answers = random.sample(not_roots, 3)
print(wrong_answers)

[{
  "id": 2,
  "text": "στρατηγῷ",
  "lemma": "στρατηγός",
  "upos": "NOUN",
  "xpos": "Nb",
  "feats": "Case=Dat|Gender=Masc|Number=Sing",
  "head": 3,
  "deprel": "iobj",
  "start_char": 3,
  "end_char": 11
}, {
  "id": 4,
  "text": "τοὺς",
  "lemma": "ὁ",
  "upos": "DET",
  "xpos": "S-",
  "feats": "Case=Acc|Definite=Def|Gender=Masc|Number=Plur|PronType=Dem",
  "head": 5,
  "deprel": "det",
  "start_char": 19,
  "end_char": 23
}, {
  "id": 1,
  "text": "τῷ",
  "lemma": "ὁ",
  "upos": "DET",
  "xpos": "S-",
  "feats": "Case=Dat|Definite=Def|Gender=Masc|Number=Sing|PronType=Dem",
  "head": 2,
  "deprel": "det",
  "start_char": 0,
  "end_char": 2
}]


    Step 4: Put all the potential answers into an array, mix them up

In [175]:
answers = []
answers.append(root[0].text)

for word in wrong_answers:
    answers.append(word.text)
    
random.shuffle(answers)
    
print(answers)

['στρατηγῷ', 'τοὺς', 'πέμπει', 'τῷ']


    Step 5: Get input

In [176]:
def check(answer):
    if answer == root[0].text:
        return "Correct!"
    return "Not quite"

demo = gr.Interface(description="What is the main verb of the sentence:", fn=check, inputs=[gr.Radio(answers, label=sentence)], outputs="text")

demo.launch()

Running on local URL:  http://127.0.0.1:7899/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fa9b2392340>, 'http://127.0.0.1:7899/', None)

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


---
### Exercise 2: Accents (identify the word(s) with the impossible accents)

NOTE: The chosen_words array may vary in length. Ultimately, it would make more sense to have it be a consistent length, even if the number of right/wrong answers changes. 

    Step 0: Place the words into a list

In [157]:
# define the file name here:
word_file = 'lib/words.txt'

# list to hold each line of the file
all_words = '[]'

# Read in the lines from the file
with open(word_file) as f:
    # create list for holding the exercises
    all_words = f.read().splitlines() 
    
print(all_words)

['ἔδεισεν ', 'δὲ ', 'βοῶπις ', 'πότνια ']


    Step 1: Randomly choose x number of words from the list. (These are the words which will be in the question)

In [158]:
# define x here 
x = 4

In [159]:
# randomly generate x number of words
chosen_words = random.sample(all_words, x)

    Step 2: Randomly split the array into two parts (correct and incorrect answers)

In [165]:
divider = random.randint(0, x)
correct_words = chosen_words[:divider]
incorrect_words = chosen_words[divider:]

print(incorrect_words)
print(correct_words)

['πότνια ', 'δὲ ']
['ἔδεισεν ', 'βοῶπις ']


    Step 3: For each 'incorrect' word,
    1. make a list of all the potentially correct accentuations
    2. strip the accents
    3. randomly insert accents
    4. check whether the accents are valid (whether they're in the list)
    5. try to replace the word with a version containing invalid accents

In [166]:
to_remove = []

# loop through each of the words for which we will assign incorrect accents
for w_index in range(0, len(incorrect_words)):
    
    # get the word, break it into syllables
    w = incorrect_words[w_index]
    s = syllabify(strip_accents(w))
    
    # types of accents
    types = [Accentuation.OXYTONE, Accentuation.PERISPOMENON, Accentuation.PAROXYTONE, Accentuation.PROPERISPOMENON, Accentuation.PROPAROXYTONE]
    
    # make a list of *potentially correct* accentuations for that word
    possible_correct = []
    for a in possible_accentuations(s):
        possible_correct.append(add_accentuation(s, a))

    # make a list of *all* the possible accentuations for that word
    all_accents = []
    for acc_type in types:
        try:
            all_accents.append(add_accentuation(s, acc_type))
        except:
            continue

    # find the difference between the two lists
    possible_incorrect = list(set(all_accents)-set(possible_correct))
    
    # if there are no incorrect answers, remove the word from incorrect_words
    if len(possible_incorrect) == 0:
        to_remove.append(w)
        
    # if there is an incorrect answer, add it to the list
    elif len(possible_incorrect) == 0:
        incorrect_words[w_index] = possible_incorrect[0]
        
    # if there is ore than one incorrect answer, randomly pick one
    else:
        incorrect_words[w_index] = random.choice(possible_incorrect)
    
    print(possible_correct)
    print(all_accents)

for r in to_remove:
    incorrect_words.remove(r)
    
print(incorrect_words)
    



['ποτνιά ', 'ποτνιᾶ ', 'ποτνία ', 'ποτνῖα ', 'πότνια ']
['ποτνιά ', 'ποτνιᾶ ', 'ποτνία ', 'ποτνῖα ', 'πότνια ']
['δέ ']
['δέ ', 'δε͂ ']
['δε͂ ']


    Step 4: shuffle the array of answers (both correct and incorrect)

In [167]:
# reinsert the incorrect answers into the original array
chosen_words = correct_words[:] + incorrect_words[:]

# shuffle the array
random.shuffle(chosen_words)

    Step 5: Get input

In [168]:
def check2(answer):
    if not len(answer) == len(incorrect_words):
        return "You have either selected too many or too few answers"
        
    for i in answer:
        if i not in incorrect_words:
            return "One or more of your answers is incorrect"
        
    return "Correct!"

demo2 = gr.Interface(description="Which of the following words contain impossible accents?:", fn=check2, inputs=[gr.CheckboxGroup(choices=chosen_words, type="value")], outputs="text")

demo2.launch()

Running on local URL:  http://127.0.0.1:7898/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fa9b239f370>, 'http://127.0.0.1:7898/', None)

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


---
### Exercise 2: Accents 